In [1]:
import spacy
from spacy.symbols import nsubj, nsubjpass, det, dobj, pobj, prep, root, neg, agent, cc, conj, acl, xcomp, punct, VERB
import io
import datetime
import json
import codecs
import csv
import re
import uuid
import pandas as pd
import numpy as np
import csv

#------------------------------------- Load data ---------------------------------------#

def load_csv_to_dict(csvfile, language, language_position):
    """
     Creates a dictionary with word tokens as keys, and their 'features' as values.
    """
    dictionary = dict()
    data = [row for row in csv.reader(open('%s' % csvfile, encoding='utf-8-sig'), delimiter=";")]
    header = data.pop(0)       # first row of csv = header.
    header.pop(0)       # remove first element of header.
    for row in data:
        if (language in row[language_position].lower()) or (language == "all"):
            key = row.pop(0)    # first element in row = key
            for index, element in enumerate(row):
                dictionary.setdefault(key.lower(), {})[header[index].lower()] = element.lower()
    return (dictionary)


def setup_spacy(user_text, nlp):
    """
     Lowers the user text and sets the spacy pipeline.
    """
    user_text = user_text.lower()
    return nlp(user_text)

In [2]:
#------------------------------------- Filter input data ---------------------------------------#

def check_user_highlight(user_highlights, datatype, target):
    """
     This function checks if a target annotation already exists in the user_highlights.
     Best result is exact matches. Otherwise it checks for match in index intervals and texts.
    """
    match = False
    interval_match = False
    text_match = False
    if datatype in user_highlights.keys():  # Check if the datatype exists.
        for key, value in user_highlights[datatype].items():
            if datatype == 'activities':  # Activities are nested in the dict.
                val_start, val_end = int(value['label']['index'][0]), int(value['label']['index'][1])  # Set index boundaries.
                tar_start, tar_end = int(target['verb']['index'][0]), int(target['verb']['index'][1]) 
                
                if 'text' in target['verb'].keys():  # Check if it contains text.
                    target_text = target['verb']['text']
                    value_text = value['label']['text']
            else:  # All other datatypes:
                val_start, val_end = int(value['index'][0]), int(value['index'][1])  # Set index boundaries.
                tar_start, tar_end = int(target['index'][0]), int(target['index'][1]) 
                
                if 'text' in target.keys():  # Check if it contains text.
                    target_text = target['text']
                    value_text = value['text']

            if (val_start, val_end) == (tar_start, tar_end):  # Checking if exact matches.
                match = True

                if target_text is not None:  # Failsafe: Checking if the strings are identical.
                    error = 'Match Error: Index match, but strings are different: {}'.format((target_text, value_text))
                    assert (target_text == value_text), error
                  
            else:   # Checks if the two intervals overlap.
                t_ran = range(tar_start, tar_end+1)
                v_ran = range(val_start, val_end+1)

                if len(list(set(t_ran) & set(v_ran))) >= 1:   
                    match = True

                    if target_text is not None:  # Failsafe: Checking string intervals.
                        error = 'Match Error: Index match, but strings are different: {}'.format((target_text, value_text))
                        assert (target_text in value_text or value_text in target_text), error

    return match

In [3]:
#------------------------------------- Entity Recognition ---------------------------------------#

def find_match_dict(user_text, dicts, user_highlights, reinforced_highlights, nlp):
    """
     This function loops over all word tokens within the user text,
     to find a match with a key within a set of dictionaries.
     It returns a dictionary with the matches string possition and attributes.
    """
    doc = setup_spacy(user_text, nlp)
    results = dict()
    for key in dicts.keys():    # iterates over the three datatypes.
        part_dict = dict()
        sent_counter = 0
        for sent in doc.sents:
            for token in sent:       # iterates over all spacy tokens in the text.
                if token.text in dicts[key] or token.lemma_ in dicts[key]:
                    id = uuid.uuid4()
                    if check_user_highlight(user_highlights, 
                                            key, 
                                            target = {"text": token.text, 
                                                      "index" : (token.idx, 
                                                                 token.idx+len(token.text))}) == True:
                        reinforced_highlights[str(id)] = {"text": token.text, 
                                                          "index" : (token.idx, token.idx+len(token.text))}
                    else:
                        noun_update = False
                        for chunk in doc.noun_chunks:
                            if (((chunk.root.idx == token.idx) and (chunk.text != token.text)) and (key == "roles")):
                                noun_update = True
                                part_dict[str(id)] = {"text": chunk.text,
                                                      "index" : (chunk.start_char, chunk.end_char), 
                                                      "sent": sent_counter}
                        
                        if noun_update == False:
                            part_dict[str(id)] = {"text": token.text,
                                                  "index" : (token.idx, token.idx+len(token.text)), 
                                                  "sent": sent_counter}
            sent_counter = sent_counter+1
        results[key] = part_dict
    return results

In [4]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [5]:
#------------------------------------- Activity Recognition ---------------------------------------#


def voice_detector(token):
    """
    Function for identifying Voice type
    Identifying passive or active voice in a given sentence.
    """
    doc = token.doc
    idx = token.i

    # passive voice:
    if token.tag_ == 'VBN' and (doc[idx-1].lemma_ == 'be' or
                                doc[idx-2].lemma_ == 'be' or
                                doc[idx-1].lemma_ == 'have'):
        voice = 'passive'
        return voice

    # active voice:
    elif token.dep_ == "ROOT" or (token.tag_ == 'VBG' and (token.dep != prep and 
         token.dep != acl)) or token.tag_ == 'VBD' or (token.tag_ == 'VBZ' 
         and token.dep_ != 'aux'):
        
        voice = 'active'
        return voice
    
    else:
        return None
    
def instance_check(idx):
    """
    Check whether index is of datatype int. 
    """
    if isinstance(idx, int):
        return idx
    else:
        return int(idx)
    
def dependencies_check(children_deps):
    """
    Check dependencies of a token's children.
    """
    deps = [nsubj, nsubjpass, dobj]
    count = 0
    for child_dep in children_deps:
        if child_dep in deps:
            count += 1
    return count

def extract_indexes(token, voice):
    """
    Exctract index span for activity text.
    """
    indexes = list()
    children = [child for child in token.children]
    children_deps = [child.dep for child in token.children]
    length = len(children)
    
    for idx, child in enumerate(children):
        
        if token.dep != conj:
            count = dependencies_check(children_deps)
            if count == 0: # if there are no children (dependency tags), break the loop
                break
        
        if voice == 'active':
            if child.dep == nsubj:
                start_idx = instance_check(child.i + 1) # don't include subject in span
                indexes.append(start_idx)
            elif child.dep == dobj:
                end_idx = instance_check(child.i + 1)
                indexes.append(end_idx)
                break
        elif voice == 'passive':
            if child.dep == nsubj:
                start_idx = instance_check(child.i + 1)
                indexes.append(start_idx) # don't include subject in span
            elif child.dep == nsubjpass:
                start_idx = child.i # include object in span (nsubjpass in passive voice is an object and not a subject)
                indexes.append(start_idx)
            elif child.dep == agent: # corresponding pobj is the actual subject in a passive voice sentence
                end_idx = child.i
                indexes.append(end_idx)
                break
            elif child.dep == dobj:
                end_idx = instance_check(child.i + 1)
                indexes.append(end_idx)
                break
        
        if idx == length - 1 and (child.dep == conj and children[idx-1].dep != cc):
            # handle conjuncts separately (as their own activities), 
            # if the previous token is not a conjunction (e.g., or, and)
            pass
        elif idx == length - 1 and (child.dep == dobj or (child.dep == conj and children[idx-1].dep == cc) or 
            child.dep == xcomp):
            end_idx = instance_check(child.i + 1)
            indexes.append(end_idx)
        elif idx == length - 1 and child.dep != dobj:
            end_idx = child.i
            indexes.append(end_idx)
    
    try:
        assert len(indexes) <= 2 # indexes list shall not contain more than two integers
    except:
        print("Problem occurred for {}: Indexes list contains too many elements (> 2)".format(token))
        
    return indexes 


def create_act_desc(token, voice):
    """
    Takes a token and a voice type (passive or active) and returns a dictionary consisting of an activity text 
    and its corresponding indexes.
    """
    act_text = None
    doc = token.doc
    idx = token.i
    indexes = extract_indexes(token, voice)

    if indexes != list():
        # create activity description
        if len(indexes) == 1:
            if indexes[0] < idx:
                start_idx = indexes[0]
                end_idx = idx+1
                act_text = doc[start_idx:end_idx]
            elif indexes[0] > idx:
                start_idx = idx
                end_idx = indexes[0]
                act_text = doc[start_idx:end_idx]

        elif len(indexes) >= 2: 
            indexes = sorted(indexes, reverse = False) # sort in ascending order 
            start_idx = indexes[0]
            end_idx = indexes[1]
            if end_idx <= idx:
                act_text = doc[start_idx:idx+1]
            else:
                act_text = doc[start_idx:end_idx]
        
    return {"text": str(act_text), "index": (act_text[0].idx, act_text[0].idx+len(str(act_text)))}
    
        
    


def process_token(token):
    """
    Given a token, its important attributes are returned.
    Returnes a set-dictionary.
    """
    prep = False
    neg = False
    children = [child for child in token.children]
    for child in children:

        if child.dep_ == "prep" or child.dep_ == "advcl":
            prep = True
        if child.dep_ == "neg":
            neg = True

    conjuncts = token.conjuncts
    
    if neg == True:
        return {"text": token.text,
            "negated": neg,
            "index": (token.idx, token.idx+len(token.text))}
    else:
        return {"text": token.text,
                "index": (token.idx, token.idx+len(token.text))}


def identify_act(token, sent):
    """
    Find object and subject for activity.
    Return token values, activity voice and activity description.
    """
    act_stat = False # used to check if there has been found an activity.
    act = {}
    # Find if activty is passive or active voice:
    act["voice"] = voice_detector(token)
    # Get token values for activity word:
    act["verb"] = process_token(token)
    act["sent"] = sent
    # passive voice handling:
    if act["voice"] == 'passive':
        for child in token.children:
            if child.dep == nsubj:
                act["subject"] = process_token(child)
                act_stat = True

            elif child.dep == nsubjpass or child.dep == dobj:
                act["object"] = process_token(child)
                act_stat = True

    # active Voice handling:
    elif act["voice"] == 'active':
        for child in token.children:
            if child.dep == nsubj:
                act["subject"] = process_token(child)
                act_stat = True

            elif child.dep == dobj:
                act["object"] = process_token(child)
                act_stat = True
    else:
        pass

    if act_stat == True:
        act["activity_label"] = create_act_desc(token, act["voice"])
        return act
    else:
        return None


def process_sent(sent, idx, user_highlights, reinforced_highlights):
    """
    Checks a sentence, if it meets the criteria for having one or more activities.
    Takes a sent, and returns a dictionary of activities.
    """
    acts_stat = False
    acts = {}
    for token in sent:
        if token.pos_ == 'VERB': 
            # If a verb is found, try and construct activity
            activity = identify_act(token, idx)
            id = uuid.uuid4()
            if identify_act(token, sent) != None:
                if check_user_highlight(user_highlights, 'activities', activity) == True:
                    reinforced_highlights[str(id)] = {"text": token.text, "index" : (token.idx, token.idx+len(token.text))}
                else:
                    acts[str(id)] = activity
                    acts_stat = True
    if acts_stat == True:
        return acts
    else:
        return None


def activity_recognition(desc, user_highlights, reinforced_highlights, nlp):
    """
    Top iteration, taking care of the document is split into sentences.
    Takes a text, and returns all identified activities not already highlighted by the user.
    Output is structured in 3 layers: 1. Sentences ( 2. Activities ( 3. Tokens)
    """
    doc = setup_spacy(desc, nlp)
    sent_acts = {}

    #sents = list(doc.sents)
    for i, sent in enumerate(doc.sents):
        analysis = process_sent(sent, i, user_highlights, reinforced_highlights)
        if analysis != None:
            sent_acts = merge_two_dicts(sent_acts, analysis)

    return sent_acts

In [6]:
#------------------------------------- API & Data Input ---------------------------------------#

# load data from csv into dictionaries:
languages = ["all", "en", "da", "pt"] # all = all languages
datatypes = [["roles", 3], ["relations", 3], ["alias", 1]]  # type, language_position
datasets = dict()
for language in languages:
    temp = dict()
    for datatype in datatypes:
        temp[datatype[0]] = load_csv_to_dict("%s.csv" % datatype[0], 
                                             language, 
                                             datatype[1])
    datasets[language] = temp
    
    
def postJsonHandler(event, context):
    """
    Handles the API call created by AWS API Gateway.
    """

    nlp = spacy.load('en_core_web_sm')

    plain_text = event["plain_text"] # Handles the input request, by breaking it into parts. 
    user_language = event["language"]
    user_highlights = event["highlights"]


    reg = re.findall(r"^\w+",user_language) # language handling:
    if reg and reg[0] in languages:
        reg_language = reg[0]

    dict_results = {}   # structures results in a combined dictionary:
    reinforced_highlights = dict()
    dict_results["reinforced_highlights"] = dict()
    dict_results["entity_recognition"] = find_match_dict(plain_text, 
                                                         datasets[reg_language], 
                                                         user_highlights, 
                                                         reinforced_highlights, 
                                                         nlp)
    if reg_language == "en" or user_language == "all":
        activities = activity_recognition(plain_text, user_highlights, reinforced_highlights, nlp)
        dict_results["activity_recognition"] = activities
        #dict_results["reinforced_highlights"] = activities[1]

    return (dict_results)
        
        
        
        
        

### API calls 

In [7]:
input = {  
    "graphid": "0",
    "userid": "0",
    "organizationid": "0",
    # graphid, userid and organizationid is not currently affecting the results.
    
    "language": "en-US",
    # the input for language loads different models. Currently we support en-*, da-*, Pt-*, and have an all option for development.
        
    "plain_text": "If the employee signs the paper work, another employee sends an email to the office. Meanwhile an Office Manager reads the email.",
    # A plain text in UTF-8. Make sure there are no strange characters and spelling mistakes.
        
    "highlights": {'roles': {0: {'text': 'employee', 'index': (7, 15)}}, 
                   'relations': {0: {'text': 'if', 'index': (0, 10)}}, 
                   'activities': {0: {'root': {'text': 'send', 'index': (50, 55)}, 
                                      'label': {'text': 'sends an email', 'index': (55, 69)}}}} 
    #Highlights are grouped in types, and each each has an id (dict key). The index is mandatory, and text is optional as a failsafe. 
    # For activities, the label is mandatory and the root is optional. However the root is most important, and therefore if the root
    # is not given, the label is analyzed in order to find the root.
    }

postJsonHandler(input, ())

{'reinforced_highlights': {},
 'entity_recognition': {'roles': {'4ded9f4a-2e2c-4204-acdb-16e2f4f0d740': {'text': 'another employee',
    'index': (38, 54),
    'sent': 0},
   '7b56978a-d598-475a-b1f5-d600f29ecbf3': {'text': 'an office manager',
    'index': (95, 112),
    'sent': 1}},
  'relations': {'faecf95b-c2cc-485d-9ab0-6414ba061c87': {'text': 'meanwhile',
    'index': (85, 94),
    'sent': 1}},
  'alias': {}},
 'activity_recognition': {'f108b920-0de7-44ba-b18a-b4853ba3f1f6': {'voice': 'active',
   'verb': {'text': 'signs', 'index': (16, 21)},
   'sent': 0,
   'subject': {'text': 'employee', 'index': (7, 15)},
   'object': {'text': 'work', 'index': (32, 36)},
   'activity_label': {'text': 'signs the paper work', 'index': (16, 36)}},
  '0f994251-00bb-423f-b1f6-2ff7eaf1498d': {'voice': 'active',
   'verb': {'text': 'reads', 'index': (113, 118)},
   'sent': 1,
   'subject': {'text': 'manager', 'index': (105, 112)},
   'object': {'text': 'email', 'index': (123, 128)},
   'activity_lab

### Benchmarking Scripts

In [201]:
dictionary = dict()
csvfile = "ex_processes.csv"
data = [row for row in csv.reader(open('%s' % csvfile, encoding='utf-8-sig'), delimiter=",")]
header = ["title", "text"]
for num, row in enumerate(data):
    key = num    # first element in row = key
    for index, element in enumerate(row):
        dictionary.setdefault(key, {})[header[index].lower()] = element.lower()

In [126]:

#------------------------------------- Generate results from dataset ---------------------------------------#

for id in dictionary:
    input = {"graphid": str(id),
             "userid": "0",
             "organizationid": "0",
             "language": "en-US",    
             "plain_text": dictionary[id]['text'],
             "highlights": {}
            }
    
    output = postJsonHandler(input, ())
    
    # Handling Output - Create 3 csv files.
    types = ["roles", "relations", "activities"]
    
    for ty in types:
        if ((ty == "roles") or (ty == "relations")):
            data = output["entity_recognition"][ty]
            
            with open('external-{}.csv'.format('{}-{}'.format(str(id), ty)), 'a') as csv_file:
                fieldnames = ['text', 'start', 'end']
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                for key in data:
                    writer.writerow({'text': data[key]["text"], 
                                     'start': data[key]["index"][0], 
                                     'end': data[key]["index"][1]})
        else:
            data = output["activity_recognition"]
            
            with open('external-{}.csv'.format('{}-{}'.format(str(id), ty)), 'a') as csv_file:
                fieldnames = ['text', 'start', 'end']
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                for key in data:
                        writer.writerow({'text': data[key]["activity_label"]["text"], 
                                         'start': data[key]["activity_label"]["index"][0], 
                                         'end': data[key]["activity_label"]["index"][1]})


Problem occurred for finished: Indexes list contains too many elements (> 2)
Problem occurred for finished: Indexes list contains too many elements (> 2)


In [405]:
def load_data(filename, delimiter=","):
    """Given a filename, data is loaded and returned in a list."""
    try:
        data = [row for row in csv.reader(open('%s' % filename, encoding='utf-8-sig'), delimiter=delimiter)]
    except:
        data = []
    return data


def overlap(start1, end1, start2, end2):
    """Check if the range (start1, end1) overlap with (start2, end2)"""
    return (end1 >= start2) and (end2 >= start1)


def compare_results(A, B):
    """Function that takes two lists with row(text, start index, end index) and compares indexes."""
    matches = []
    A_leftovers = []

    # Compare A to B:
    for A_row in A:
        if A_row[1] == '':
            pass
        else:
            A_start = int(A_row[1])
            A_end = int(A_row[2])
            match = False

            for B_row in B:
                if match == True:
                    pass
                else:
                    if B_row[1] == '':
                        pass
                    else:
                        B_start = int(B_row[1])
                        B_end = int(B_row[2]) 

                        if overlap(A_start, A_end, B_start, B_end) == True:
                            matches.append([(A_start, A_end), (B_start, B_end), A_row[0], B_row[0]])
                            match = True

        if match == False:
            A_leftovers.append([A_row[0], A_start, A_end])
    return matches


def create_statistics(list):
    """Given a list with pairs, compare each pair and return the results in a list."""
    statistics = []
    for pair in list:
        A = load_data(pair[0])
        B = load_data(pair[1])
        matches = compare_results(A, B)
        statistics.append(["{}".format(pair[0]), len(A), len(matches)])
    return statistics


def create_list_pairs(filetemps, n):
    pairs = []
    for idx in range(1,n+1):
        pairs.append([filetemps[0].format(idx), filetemps[1].format(idx)])
    return pairs


#------------------------------------- Compare two result files ---------------------------------------#


templates = ["{}-activities.csv", "JM-{}-activities-ML.csv"]

def template(ty):
    RB = "{}-" + str(ty) + ".csv"
    ML = "{}-" + str(ty) + ".csv"
    return [RB, ML]

#templates = [templates[1], templates[0]]
stats = create_statistics(create_list_pairs(template("activities"), 37))

stats_df = pd.DataFrame((stats), columns =['Filename', 'Elements in file', 'Matches'])

In [290]:
stats = create_statistics(create_list_pairs(template("roles"), 37))
stats_df = pd.DataFrame((stats), columns =['Filename', 'Elements in file', 'Matches'])


In [291]:
import xmltodict

def get_user_annos(filename, annotation_type):
    with open(filename) as fd:
        doc = xmltodict.parse(fd.read())
    output_dict = json.loads(json.dumps(doc))
    highlights = []
    lista = []
    user_annos = []
    for highlight in output_dict["dcrgraph"]["specification"]["resources"]["custom"]["highlighterMarkup"]["highlights"]["highlight"]:

        annos = [] 
        if highlight["@type"] == annotation_type:
            annos = []

            for key, value in highlight["layers"]["layer"]["ranges"]["range"].items():
                if key == "@start":
                    start = value
                elif key == "@end":
                    end = value
                elif key == "#text":
                    text = value
            anno = [text,start,end]
            user_annos.append(anno)
    return user_annos

def get_graph_name(filename):
    with open(filename) as fd:
        doc = xmltodict.parse(fd.read())
    output_dict = json.loads(json.dumps(doc))
    return output_dict["dcrgraph"]["@title"]

#------------------------------------- Compare user annotations against file ---------------------------------------#

def template(ty):
    RB = "{}-" + str(ty) + ".csv"
    ML = "JM-{}-" + str(ty) + "-ML.csv"
    return [RB, ML]

In [414]:
#------------------------------------- Compare RB + ML with dataset ---------------------------------------#

def compare_results_to_graph(file, annotation_type, method="intersection"):
    """Compare annotations for rb, ml, or the intersection between them and an annotated graph."""
    
    statistics = []
    file_name = get_graph_name(file[1])
    user_annos = get_user_annos(file[1], annotation_type[0])
    
    
    targets = ["external-{}-".format(file[0]-1) + str(annotation_type[1]) + ".csv", 
               "JM-{}-".format(file[0]) + str(annotation_type[1]) + ".csv"]
    A = load_data(targets[0])
    B = load_data(targets[1])
    matches = compare_results(A, B)
    reordered_matches = []
    for match in matches:
        text = match[2]
        start = match[0][0]
        end = match[0][1]
        reorder = [text, start, end]
        reordered_matches.append(reorder)
        
        
    if method == "intersection":    
        prediction = reordered_matches
        hits = compare_results(user_annos, prediction)
        
    elif method == "rb":
        prediction = A
        hits = compare_results(user_annos, prediction)
        
    elif method == "ml":
        prediction = B
        hits = compare_results(user_annos, prediction)
    
    gold = user_annos
    
    if len(hits) != 0:    
        
        precision = len(hits)/len(prediction)
        recall = len(hits)/len(gold)
        f1 = (2*precision*recall)/(precision+recall)
    else:
        precision = 0
        recall = 0
        f1 = 0
    statistics = ["{} {}: {}".format(file[0], file_name, ty[1]), 
                  len(prediction), 
                  len(gold), 
                  len(hits),
                  precision,
                  recall,
                  f1]
    return statistics

def Average(lst): 
    if sum(lst) != 0:
        return sum(lst) / len(lst) 
    else:
        return 0
    
def classify_sents(prediction, file):
    file_name = get_graph_name(file[1])
    gold = file[2]
    hits = len(prediction) - abs(gold - len(prediction))
    if len(prediction) != 0:    
        precision = (hits)/len(prediction)
        recall = (hits)/(gold)
        f1 = (2*precision*recall)/(precision+recall)
    else:
        precision = 0
        recall = 0
        f1 = 0
    statistics = ["{} {}: {}".format(file[0], file_name, "relations"), 
                  len(prediction), 
                  (gold), 
                  (hits),
                  precision,
                  recall,
                  f1]
    return statistics

In [446]:
files = [[1, "DE1.xml", 2], [2, "DE2.xml", 4], [3, "DE3.xml", 1], [4, "DE4.xml", 3], [5, "ATDP1.xml", 4], [6, "ATDP2.xml", 6], [7, "1.xml", 4], 
        [8, "2.xml", 5], [9, "3.xml", 6], [10, "4.xml", 7]]                                            

#test = [[1, "DE1.xml", 3]]

#user_files = [[1, "1.xml"], [2, "2.xml"], [3, "3.xml"], [4, "4.xml"], [7, "7.xml"], 
#             [13, "13.xml"], [21, "21.xml"]]     

stats = []
for file in files:
    for ty in [["activity", "activities"]]:
        stats.append(compare_results_to_graph(file, ty, "intersection"))
    for ty in [["role", "roles"]]:
        stats.append(compare_results_to_graph(file, ty, "ml"))
    rel = load_data("JM-{}-Relation.csv".format(file[0]))
    stats.append(classify_sents(rel, file))

prec = []
recall = []
f1 = []
for row in stats:
    prec.append(row[4])
    recall.append(row[5])
    f1.append(row[6])
print("Avg prec Score: " + str(Average(prec)) + "\nAvg recall Score: " + 
      str(Average(recall)) + "\nAvg F1 Score: " + str(Average(f1)))

stats_df = pd.DataFrame(stats, columns =['Filename', 'Predictions', 'Graph', 'True Positives', 'Prec', 'Recall', 'F1'])
stats_df

Avg prec Score: 0.7625258564964449
Avg recall Score: 0.7843316405816407
Avg F1 Score: 0.7196487900950376


,Filename,Predictions,Graph,True Positives,Prec,Recall,F1
0,1 DCR: activities,7,9,6,0.857143,0.666667,0.750000
1,1 DCR: roles,2,2,2,1.000000,1.000000,1.000000
2,1 DCR: relations,3,2,2,0.666667,1.000000,0.800000
3,2 van der : activities,6,12,7,1.166667,0.583333,0.777778
4,2 van der : roles,6,3,2,0.333333,0.666667,0.444444
5,2 van der : relations,4,4,4,1.000000,1.000000,1.000000
6,3 vander aa 3: activities,4,4,3,0.750000,0.750000,0.750000
7,3 vander aa 3: roles,3,2,1,0.333333,0.500000,0.400000
8,3 vander aa 3: relations,2,1,1,0.500000,1.000000,0.666667
9,4 DCR: activities,6,7,7,1.166667,1.000000,1.076923
